In [ ]:
!pip install -r requirements.txt

# Scikit-learn Linear Regression Example - Interactive API

In [ ]:
from typing import List, Union

from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 7, 5

# We will use MSE as loss function and Ridge weights regularization
![image.png](https://www.analyticsvidhya.com/wp-content/uploads/2016/01/eq5-1.png)

In [ ]:
class SklearnLinearRegressionLasso:
    def __init__(self, n_feat: int, alpha: float = 1.0) -> None:
        self.model = Lasso(alpha=alpha)
        self.scaler = StandardScaler()
        self.weights = np.ones((n_feat + 1)) 
        
    def predict(self, feature_vector: Union[np.ndarray, List[int]]) -> float:
        '''
        feature_vector may be a list or have shape (n_feat,)
        or it may be a bunch of vectors (n_vec, nfeat)
        '''
        feature_vector = np.array(feature_vector)
        if len(feature_vector.shape) == 1:
            feature_vector = feature_vector[:,np.newaxis]
            
        feature_vector = self.scaler.transform(feature_vector)
        return self.model.predict(feature_vector)
    
    def mse(self, X: np.ndarray, Y: np.ndarray) -> float:
        Y_predict = self.predict(X)
        return mean_squared_error(Y, Y_predict)
    
    def fit(self, X: np.ndarray, Y: np.ndarray, silent: bool=False) -> None:
   
        X = self.scaler.fit_transform(X)
        self.model.fit(X, Y)
        mse = self.mse(X, Y)
        #self.weights[:-1] = self.model.coef_
        #self.weights[-1] = self.model.intercept_
        if not silent:
            print(f'MSE: {mse}')
            
    def print_parameters(self) -> None:
        print('Final parameters: ')
        print(f'Weights: {self.model.coef_}')
        print(f'Bias: {self.model.intercept_}')
            

In [ ]:
# Define input array with angles from 60deg to 300deg converted to radians
x = np.array([i*np.pi/180 for i in range(60,300,4)])
np.random.seed(10)  # Setting seed for reproducibility
y = np.sin(x) + np.random.normal(0,0.15,len(x))
# plt.plot(x,y,'.')

In [ ]:
lr_model = SklearnLinearRegressionLasso(n_feat=1, alpha=0.1)

lr_model.fit(x[:,np.newaxis], y)

#print the final parameters
lr_model.print_parameters()

In [ ]:
# We can also solve this 1D problem using Numpy
numpy_solution = np.polyfit(x,y,1)
predictor_np = np.poly1d(numpy_solution)

In [ ]:
y_hat = lr_model.predict(x)
y_np = predictor_np(x)
plt.plot(x,y,'.')
plt.plot(x,y_hat,'.')
plt.plot(x,y_np,'--')

# Now we run the same training on federated data

## Connect to a Federation

In [ ]:
# Create a federation
from openfl.interface.interactive_api.federation import Federation

# please use the same identificator that was used in signed certificate
client_id = 'frontend'
director_node_fqdn = 'localhost'
director_port = 50050

federation = Federation(
    client_id=client_id,
    director_node_fqdn=director_node_fqdn,
    director_port=director_port,
    tls=False
)

In [ ]:
shard_registry = federation.get_shard_registry()
shard_registry

### Data

In [ ]:
from openfl.interface.interactive_api.experiment import TaskInterface, DataInterface, ModelInterface, FLExperiment

class LinRegDataSet(DataInterface):
    def __init__(self, **kwargs):
        """Initialize DataLoader."""
        self.kwargs = kwargs
        pass

    @property
    def shard_descriptor(self):
        """Return shard descriptor."""
        return self._shard_descriptor
    
    @shard_descriptor.setter
    def shard_descriptor(self, shard_descriptor):
        """
        Describe per-collaborator procedures or sharding.

        This method will be called during a collaborator initialization.
        Local shard_descriptor  will be set by Envoy.
        """
        self._shard_descriptor = shard_descriptor
        self.train_set = shard_descriptor.get_dataset("train")
        self.val_set = shard_descriptor.get_dataset("val")

    def get_train_loader(self, **kwargs):
        """Output of this method will be provided to tasks with optimizer in contract."""
        return self.train_set

    def get_valid_loader(self, **kwargs):
        """Output of this method will be provided to tasks without optimizer in contract."""
        return self.val_set

    def get_train_data_size(self):
        """Information for aggregation."""
        return len(self.train_set)

    def get_valid_data_size(self):
        """Information for aggregation."""
        return len(self.val_set)
    
lin_reg_dataset = LinRegDataSet()

### Model

In [ ]:
framework_adapter = 'custom_adapter.CustomFrameworkAdapter'
fed_model = SklearnLinearRegressionLasso(n_feat=1, alpha=0.1)
MI = ModelInterface(model=fed_model, optimizer=None, framework_plugin=framework_adapter)

# Save the initial model state
initial_model = SklearnLinearRegressionLasso(n_feat=1, alpha=0.1)

### Tasks
We need to employ a trick reporting metrics. OpenFL decides which model is the best based on an *increasing* metric.

In [ ]:
TI = TaskInterface()

@TI.register_fl_task(model='my_model', data_loader='train_data', \
                     device='device', optimizer='optimizer')     
def train(my_model, train_data, optimizer, device):
    X, Y = train_data[:,:-1], train_data[:,-1]
    my_model.fit(X, Y, silent=True)
    return {'train_MSE': my_model.mse(X, Y),}

@TI.register_fl_task(model='my_model', data_loader='val_data', device='device')     
def validate(my_model, val_data, device):
    X, Y = val_data[:,:-1], val_data[:,-1]        
    return {'validation_MSE': my_model.mse(X, Y),}

### Run

In [ ]:
experiment_name = 'linear_regression_experiment'
fl_experiment = FLExperiment(federation=federation, experiment_name=experiment_name,
                            )

In [ ]:
fl_experiment.start(model_provider=MI, 
                    task_keeper=TI,
                    data_loader=lin_reg_dataset,
                    rounds_to_train=10,)

In [ ]:
fl_experiment.stream_metrics()